In [1]:
import pandas as pd
import geopandas as gp
from src.utils.map_vis import geojson_per_row
import folium

In [2]:
num_cities = 16

# Colombia

In [3]:
shp_munic = gp.read_file('data/colombia/ADMINISTRATIVO/MGN_MPIO_POLITICO.shp')
col_munic = ['DPTO_CCDGO','MPIO_CCDGO', 'MPIO_CNMBR', 'DPTO_CNMBR', 'geometry']

# CLAS_CCDGO: 1-Municipal Headboard 2-populated center 3-scattered rural area 
# 1-Cabecera municipal, 2-Centro poblado y 3-Área rural dispersa.
shp_urb_censal = gp.read_file('data/colombia/MGN/MGN_URB_AREA_CENSAL.shp')
print(shp_urb_censal.shape)
shp_urb_censal=shp_urb_censal.merge(shp_munic[col_munic[:-1]])
shp_urb_censal.head()

(7627, 14)


,OBJECTID,DPTO_CCDGO,MPIO_CCDGO,CLAS_CCDGO,SETR_CCDGO,SECR_CCDGO,CPOB_CCDGO,CPOB_CNMBR,CPOB_CCNCT,CPOB_NAREA,CPOB_NALT,Shape_Leng,Shape_Area,geometry,MPIO_CNMBR,DPTO_CNMBR
0,1,05,05001,2,000,00,05001004,SANTA ELENA,05001200000004,0.338461,2540.0,0.034407,0.000028,POLYGON ((-75.49783904499998 6.207774365000034...,MEDELLÍN,ANTIOQUIA
1,2,05,05001,2,000,00,05001014,BARRO BLANCO,05001200000014,0.534495,2531.0,0.044447,0.000044,POLYGON ((-75.47379840599996 6.239590113000077...,MEDELLÍN,ANTIOQUIA
2,3,05,05001,2,000,00,05001015,EL CERRO,05001200000015,0.414935,2668.0,0.042879,0.000034,(POLYGON ((-75.49121943999995 6.20501584900006...,MEDELLÍN,ANTIOQUIA
3,4,05,05001,2,000,00,05001018,EL PLACER,05001200000018,0.965299,2598.0,0.056312,0.000079,"POLYGON ((-75.49276999399996 6.23050003000003,...",MEDELLÍN,ANTIOQUIA
4,5,05,05001,2,000,00,05001019,EL PLAN,05001200000019,3.252561,2700.0,0.093795,0.000266,POLYGON ((-75.50917742799999 6.195200505000059...,MEDELLÍN,ANTIOQUIA


In [4]:
population = pd.read_csv('data/colombia/population_2005.csv',sep='\t')
cols = ['TOTAL_2005','CABECERA_2005', 'RESTO_2005']
for col in cols:
    population[col] = population[col].apply(lambda x: float(x.replace(',', '')))
    
population['CABECERA_pcnt'] = population.CABECERA_2005/population.TOTAL_2005
population.sort_values('TOTAL_2005', ascending=False, inplace=True)
population['pop_order'] = pd.np.arange(population.shape[0])+1
population.DPMP = population.DPMP.apply(lambda x: '%05d' % x)

In [5]:
thres = 0.8
target_munic = population[population.CABECERA_pcnt>thres].head(num_cities)

In [6]:
target_cities = shp_urb_censal.merge(target_munic, left_on='MPIO_CCDGO', right_on='DPMP')
target_cities = target_cities[target_cities.CLAS_CCDGO=='1'].copy()

cols = ['OBJECTID', 'DPTO_CCDGO', 'MPIO_CCDGO', 'CLAS_CCDGO', 'CPOB_CCDGO', 'MPIO_CNMBR', 'MPIO', 'CPOB_CNMBR', 'CPOB_NAREA', 'TOTAL_2005',
       'CABECERA_2005', 'RESTO_2005', 'CABECERA_pcnt', 'pop_order', 'geometry']

target_cities = target_cities[cols]

In [7]:
target_cities.to_file('data/cities_colombia_16.geojson', driver="GeoJSON")

In [8]:
some_map = folium.Map(location=[4.149624, -72.795576], zoom_start=6)
geojson_per_row(target_cities, name='cities', tip_cols=['CPOB_CCDGO', 'MPIO_CNMBR', 'MPIO', 'CPOB_CNMBR'], some_map=some_map)
folium.LayerControl().add_to(some_map)
some_map.save('data/cities_colombia_16.html')

# Mex

In [9]:
from src.utils import fix_spanish_encoding

In [10]:
# http://www.inegi.org.mx/geo/contenidos/geoestadistica/m_g_0.aspx
shp_urban = gp.read_file('data/mexico/mglu2014v6_2/mglu2015v6_2.shp')
shp_state = gp.read_file('data/mexico/mge2014v6_2/mge2015v6_2.shp')
shp_munic = gp.read_file('data/mexico/mgm2014v6_2/mgm2015v6_2.shp')

# get Urban id
shp_urban.NOM_LOC = shp_urban.NOM_LOC.apply(fix_spanish_encoding)
shp_urban['URBAN_ID'] = shp_urban.CVE_ENT + shp_urban.CVE_MUN + shp_urban.CVE_LOC

# get the full data frame
shp_all = shp_urban.merge(shp_state.drop('geometry', axis=1)).merge(shp_munic.drop('geometry', axis=1))

### Fix the holes in mexico city

In [11]:
from shapely.ops import cascaded_union
from shapely.geometry import Polygon, MultiPolygon

In [12]:
polys = cascaded_union(shp_all[shp_all.CVE_ENT=='09'].geometry)
p0=polys[0]
p0 = Polygon(p0.exterior, [p0.interiors[115]])
p4 = Polygon(polys[4].exterior)

mex_city_geo = MultiPolygon([p0, p4]+list(polys[1:4]) + list(polys[5:]))

In [13]:
mexico_city = {'CVE_ENT': '09', 'CVE_MUN': '000', 'CVE_LOC': '0000', 
               'NOM_LOC': 'Mexico city in DC', 'NOM_ENT': 'Distrito Federal',
               'NOM_MUN': 'Mexico city in DC', 'concat': '09000', 'URBAN_ID' :'090000000',
               'geometry': mex_city_geo}
mexico_city = gp.GeoDataFrame([mexico_city])

### output geojson and map

In [14]:
idx = [742, 3396, 454, 431, 2095, 3515, 1186, 48, 2280, 1943, 3815, 2373, 71, 1771, 3709]
target_cities_mex = shp_all.loc[idx].append(mexico_city, ignore_index=True, sort=False)

cols = ['URBAN_ID', 'NOM_ENT', 'NOM_MUN','NOM_LOC',  'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'geometry']
target_cities_mex = target_cities_mex[cols]

In [15]:
target_cities_mex.to_file('data/cities_mexicol_16.geojson', driver="GeoJSON")

In [16]:
some_map = folium.Map(location=[24.062199, -102.870324], zoom_start=5)
geojson_per_row(target_cities_mex, name='cities', tip_cols=['URBAN_ID', 'NOM_ENT', 'NOM_MUN','NOM_LOC',], some_map=some_map)
folium.LayerControl().add_to(some_map)
some_map.save('data/cities_mexico_16.html')